In [8]:
import os
import json
import datetime as dt
import pandas as pd
import numpy as np
import sys
import requests
import datetime as dt

In [9]:
##get trailhead hotspot list

trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots_Y2022_M1_D6.csv')

In [10]:
ebird_token = 'j6c7l80ga2ib'

##be nice to eBird, get a filtered list from all trailhead hotspots for testing
##the code below is for testing, interchange stopNames as needed
THfilters = ['HighSchool','EastSunsetWay']
THfiltersBool = trailheadHotspots.StopName.isin(THfilters)
selectHotspots = pd.DataFrame(trailheadHotspots[THfiltersBool])

In [11]:

##get recent notable observations

notableObs_current = []

##Really, really need to switch out this loop for a function
for locId in trailheadHotspots.itertuples():
##for locId in selectHotspots.itertuples():
    ebird_url = 'https://api.ebird.org/v2/data/obs/geo/recent/notable?'
    ebird_authHeader = {'X-eBirdApiToken': ebird_token}
    ebird_params = {
        'lat': str(locId.lat),
        'lng': str(locId.lng),
        'dist': str(2),
        'back': str(30),
        'detail': 'simple',
        'fmt': 'json'
    }
    ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_authHeader)
    if ebird_request.status_code == requests.codes.ok:
        ebird_response = pd.DataFrame(ebird_request.json())
        ebird_response['StopName'] = locId.StopName
        notableObs_current.append(ebird_response)
    ebird_request.raise_for_status()

In [12]:
all_notableObs_current = pd.concat(notableObs_current)
##sanity check
all_notableObs_current.head(25)

,StopName,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395


In [13]:
##get historical data on all notable observations in a +/- 30 day window

date_range = []

today = dt.date.today()
todayBack = today - dt.timedelta(30)
todayOut = today + dt.timedelta(30)
yearStart = todayBack.year - 4
yearStop = todayOut.year - 1
monthBack = todayBack.month
monthOut = todayOut.month

for i in range(yearStart,yearStop):
    days = pd.date_range(
        start=dt.date(i,monthBack,todayBack.day),
        end=dt.date(i+1,monthOut,todayOut.day)
    )
    date_range.append(days)


In [14]:
date_list = []
for sublist in date_range:
    for item in sublist:
        date_list.append(item)

In [100]:
##get historic notable observations at the given hotspots
##TODO: #5 fix stopname loop problem

histNotableObs_atLoc = []
notable_locFrame = pd.DataFrame(all_notableObs_current)

for locId in notable_locFrame.itertuples():
    for date in date_list:
        ##time.sleep(0.5)
        ymd = '{}/{}/{}'.format(date.year,date.month,date.day)
        ebirdHist_url = 'https://api.ebird.org/v2/data/obs/' + locId.locId + '/historic/' + ymd
        ebirdHist_auth = {'X-eBirdApiToken': ebird_token}
        ebirdHist_params = {
            'fmt':'json',
            'detail':'simple'
        }
        ebirdHist_req = requests.get(ebirdHist_url,params=ebirdHist_params,headers=ebirdHist_auth)
        if ebirdHist_req.status_code == requests.codes.ok:
            ebirdHist_resp = pd.DataFrame(ebirdHist_req.json())
            ##ebirdHist_resp['StopName'] = pd.merge(ebirdHist_resp,all_notableObs_current[['StopName']],left_on='locId',right_on='locId',how='left')
            ebirdHist_req.raise_for_status()
    ebirdHist_resp['StopName'] = locId.StopName
    histNotableObs_atLoc.append(ebirdHist_resp)

In [101]:
all_histNotableObs = pd.concat(histNotableObs_atLoc)
##another sanity check
all_histNotableObs.head(15)

,StopName


In [102]:
##filter for notables
SpeciesFilters = list(all_notableObs_current['speciesCode'])
SpeciesFilters_bool = all_histNotableObs.speciesCode.isin(SpeciesFilters)
select_obs = pd.DataFrame(all_histNotableObs[SpeciesFilters_bool])

select_obs.head(25)

AttributeError: 'DataFrame' object has no attribute 'speciesCode'

In [ ]:
##clean df
select_obs['obsDt'] = select_obs['obsDt'].astype('datetime64[ns]')

##plot notables by locName

testlist = []
needPd = pd.DataFrame(all_notableObs_current)

for StopName in needPd.itertuples():
    for i in range(1,5):
        sightings = 10 + i
        sumSightings = pd.DataFrame(np.array([[i,3,4],[5,i,6],[i,i,3]]))
    sumSightings['StopName'] = StopName.StopName
    testlist.append(sumSightings)

alltestlist = pd.concat(testlist)
alltestlist.head(25)